# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [ ]:
def extract_time_series_features(data, window_size=60):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    # Your code here
    # 1. Calculate rolling window statistics
    # 2. Include mean, std, min, max, and autocorrelation
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    feature_dfs = []
    signals = ['heart_rate', 'eda', 'temperature']

    for (subject, session), group in data.groupby(['subject_id', 'session']):
        group = group.sort_values('timestamp').copy()
        group.set_index('timestamp', inplace=True)

        rolled = group[signals].rolling(f'{window_size}s', min_periods=1)

        features = pd.DataFrame(index=group.index)

        for signal in signals:
            features[f'mean_{signal}'] = rolled[signal].mean()
            features[f'std_{signal}'] = rolled[signal].std()
            features[f'min_{signal}'] = rolled[signal].min()
            features[f'max_{signal}'] = rolled[signal].max()
            features[f'autocorr_{signal}'] = rolled[signal].apply(
                lambda x: x.autocorr(lag=1) if len(x.dropna()) > 1 else np.nan
            )

        features = features.reset_index()
        features['subject_id'] = subject
        features['session'] = session

        feature_dfs.append(features)

    return pd.concat(feature_dfs, ignore_index=True)

In [15]:
df_clean = pd.read_csv('/Users/jessica/4-it-s-about-time-hojess20/data/processed/physiological_data_cleaned.csv')
features_df = extract_time_series_features(df_clean, window_size=60)
features_df.head()

/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dty

,timestamp,mean_heart_rate,std_heart_rate,min_heart_rate,max_heart_rate,autocorr_heart_rate,mean_eda,std_eda,min_eda,max_eda,autocorr_eda,mean_temperature,std_temperature,min_temperature,max_temperature,autocorr_temperature,subject_id,session
0,2018-12-05 16:29:08,116.000000,NaN,116.0,116.0,NaN,0.023064,NaN,0.023064,0.023064,NaN,21.910000,NaN,21.91,21.91,NaN,S1,Final
1,2018-12-05 16:29:09,99.250000,23.688077,82.5,116.0,NaN,0.023064,0.000000,0.023064,0.023064,NaN,21.920000,0.014142,21.91,21.93,NaN,S1,Final
2,2018-12-05 16:29:10,98.276667,16.834626,82.5,116.0,-1.000000,0.023491,0.000740,0.023064,0.024345,NaN,21.923333,0.011547,21.91,21.93,NaN,S1,Final
3,2018-12-05 16:29:11,95.270000,15.003220,82.5,116.0,-0.936368,0.023704,0.000740,0.023064,0.024345,0.500000,21.920000,0.011547,21.91,21.93,-5.000000e-01,S1,Final
4,2018-12-05 16:29:12,95.936000,13.078235,82.5,116.0,-0.909283,0.023320,0.001072,0.021783,0.024345,-0.301511,21.918000,0.010954,21.91,21.93,-3.138765e-17,S1,Final


## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [ ]:
def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Drop NaNs
    series = series.dropna().astype(float)

    # Fit ARIMA model
    model = ARIMA(series, order=order)
    fitted_model = model.fit()

    # Actual vs Fitted
    plt.figure(figsize=(10, 4))
    plt.plot(series, label='Original')
    plt.plot(fitted_model.fittedvalues, label='Fitted', linestyle='--')
    plt.title('ARIMA Model Fit')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_fit.png'))
    plt.close()

    # Residuals plot
    residuals = fitted_model.resid
    plt.figure(figsize=(10, 4))
    plt.plot(residuals)
    plt.axhline(0, color='gray', linestyle='--')
    plt.title('Residuals')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_residuals.png'))
    plt.close()

    # Forecast
    forecast = fitted_model.get_forecast(steps=30)
    pred_mean = forecast.predicted_mean
    conf_int = forecast.conf_int()

    plt.figure(figsize=(10, 4))
    plt.plot(series, label='Original')
    plt.plot(pred_mean, label='Forecast', color='green')
    plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='green', alpha=0.2)
    plt.title('30-Step Forecast')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_forecast.png'))
    plt.close()

    return fitted_model

In [ ]:
subset = df_clean[(df_clean['subject_id'] == 'S5') & (df_clean['session'] == 'Final')]

subset = subset.sort_values('timestamp')
subset['timestamp'] = pd.to_datetime(subset['timestamp'])
subset = subset.set_index('timestamp')

test_series = subset['heart_rate']
fitted_model = build_arima_model(test_series, order=(2,1,2))
print(fitted_model.summary())

/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:             heart_rate   No. Observations:                15177
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -6092.379
Date:                Tue, 10 Jun 2025   AIC                          12194.758
Time:                        12:12:51   BIC                          12232.896
Sample:                             0   HQIC                         12207.405
                              - 15177                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0776      0.001   -146.768      0.000      -0.079      -0.077
ar.L2          0.9208      0.001   1728.980      0.000       0.920       0.922
ma.L1         -0.0475      0.002    -20.567      0.0

/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/jessica/4-it-s-about-time-hojess20/venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
